# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Lab 08** </center>

---
**Equipo**: Equipo 

**Integrantes**:
- Yael Alejandro Rodríguez Barreto

**Profesor**: Dr. Pablo Camarillo Ramirez


In [1]:
import findspark
findspark.init()

#### Creacion de la conexión con el cluster de spark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQLStructuredStreaming-Kafka") \
    .master("spark://30b7a7c111a4:7077") \
    .config("spark.ui.port","4040") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.4") \
    .getOrCreate()
sc = spark.sparkContext

:: loading settings :: url = jar:file:/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-87ad8087-7ef3-4086-a418-f2515b03a3b8;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;3.5.4 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;3.5.4 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.scala-lang.modules#scala-parallel-collections_2.13;1.0.4 in central
	found org.apache.commons#commons-pool2;2.11.1 in centr

### Creación del Kafka Stream

In [3]:
kafka_lines = spark \
                .readStream \
                .format("kafka") \
                .option("kafka.bootstrap.servers", "d10938ce62b1:9093") \
                .option("subscribe", "kafka-spark-example") \
                .load()

kafka_lines.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



### Transform binary data into string

In [4]:
kafka_df = kafka_lines.withColumn("value_str", kafka_lines.value.cast("string"))

### Configuración del "Sink" del stream

In [5]:

query = kafka_df \
                .writeStream \
                .outputMode("append") \
                .trigger(processingTime='3 seconds') \
                .format("console") \
                .option("truncate", "false") \
                .start()

query.awaitTermination(300)


25/04/04 14:07:48 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-6ced9d2a-6bb0-4aaa-afb1-da7edf7ea5f2. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/04/04 14:07:48 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/04/04 14:07:51 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+---+-----+-----+---------+------+---------+-------------+---------+
|key|value|topic|partition|offset|timestamp|timestampType|value_str|
+---+-----+-----+---------+------+---------+-------------+---------+
+---+-----+-----+---------+------+---------+-------------+---------+



25/04/04 14:07:56 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 3000 milliseconds, but spent 6983 milliseconds


-------------------------------------------
Batch: 1
-------------------------------------------
+----+-------+-------------------+---------+------+-----------------------+-------------+---------+
|key |value  |topic              |partition|offset|timestamp              |timestampType|value_str|
+----+-------+-------------------+---------+------+-----------------------+-------------+---------+
|NULL|[59 61]|kafka-spark-example|0        |1     |2025-04-04 14:07:59.866|0            |Ya       |
+----+-------+-------------------+---------+------+-----------------------+-------------+---------+



25/04/04 14:08:11 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 3000 milliseconds, but spent 8009 milliseconds


...

-------------------------------------------
Batch: 7
-------------------------------------------
+----+-------------------------------------------------------------------------+-------------------+---------+------+----------------------+-------------+------------------------+
|key |value                                                                    |topic              |partition|offset|timestamp             |timestampType|value_str               |
+----+-------------------------------------------------------------------------+-------------------+---------+------+----------------------+-------------+------------------------+
|NULL|[4C 65 73 20 66 61 6C 74 61 6E 20 62 61 63 68 65 73 20 6A 61 20 6A 61 73]|kafka-spark-example|0        |7     |2025-04-04 14:09:37.95|0            |Les faltan baches ja jas|
+----+-------------------------------------------------------------------------+-------------------+---------+------+----------------------+-------------+------------------------+

--

-------------------------------------------
Batch: 9
-------------------------------------------
+----+-------------------------------------------------------------+-------------------+---------+------+-----------------------+-------------+--------------------+
|key |value                                                        |topic              |partition|offset|timestamp              |timestampType|value_str           |
+----+-------------------------------------------------------------+-------------------+---------+------+-----------------------+-------------+--------------------+
|NULL|[55 6E 20 73 61 6C 75 64 6F 20 61 20 6C 61 20 67 72 61 73 61]|kafka-spark-example|0        |9     |2025-04-04 14:09:57.302|0            |Un saludo a la grasa|
+----+-------------------------------------------------------------+-------------------+---------+------+-----------------------+-------------+--------------------+



-------------------------------------------
Batch: 10
-------------------------------------------
+----+----------------------------+-------------------+---------+------+-----------------------+-------------+---------+
|key |value                       |topic              |partition|offset|timestamp              |timestampType|value_str|
+----+----------------------------+-------------------+---------+------+-----------------------+-------------+---------+
|NULL|[59 61 20 63 61 6D 62 69 6F]|kafka-spark-example|0        |10    |2025-04-04 14:10:15.048|0            |Ya cambio|
+----+----------------------------+-------------------+---------+------+-----------------------+-------------+---------+

-------------------------------------------
Batch: 11
-------------------------------------------
+----+-------------------------------------------------------+-------------------+---------+------+-----------------------+-------------+------------------+
|key |value                              

-------------------------------------------
Batch: 12
-------------------------------------------
+----+-------------------------------------------+-------------------+---------+------+-----------------------+-------------+--------------+
|key |value                                      |topic              |partition|offset|timestamp              |timestampType|value_str     |
+----+-------------------------------------------+-------------------+---------+------+-----------------------+-------------+--------------+
|NULL|[46 61 6C 74 61 20 31 20 6D 69 6E 75 74 6F]|kafka-spark-example|0        |12    |2025-04-04 14:11:42.816|0            |Falta 1 minuto|
+----+-------------------------------------------+-------------------+---------+------+-----------------------+-------------+--------------+



False

In [6]:
query.stop()

In [7]:
sc.stop()